# Bullet Forge Fine-tuning

This notebook is used for fine-tuning the Bullet Forge model in Open AI's API. Please refer to the README for more details.

## Step 1: Provide Required Arguments

Make sure to inject your environment variables by executing the following at the root of the repository:

```bash
# if using venv
source ~/smarter-bullets/.venv/bin/activate
# source env file
source ./config/.env.local
```

Make sure to provide the following arguments:
- JSONL_FILE: The path to the JSONL file containing the training data.
- BASE_MODEL: The base model to use for fine-tuning.

In [19]:
JSONL_FILE = input("Enter the path to the JSON file: ")
print(f"Provided JSONL file path: {JSONL_FILE}")
BASE_MODEL = input("Enter the base model: ")
print(f"OpenAI base model: {BASE_MODEL}")

Provided JSONL file path: ./example-data.jsonl
OpenAI base model: ada-003-text


## Step 2: Prepare Training Data

Prepare the training data using the provided JSON file.

In [13]:
import subprocess

print("Preparing training data...")
if not JSONL_FILE:
    exit
PREPARED_DATA = subprocess.run(['openai', 'tools', 'fine_tunes.prepare_data', '-f', JSONL_FILE], capture_output=True, text=True)
print(PREPARED_DATA.stdout)
if PREPARED_DATA.returncode == 0:
    print("Training data looks good!")
else:
    print("Failed to analyze training data.")
    print(f"Error: {PREPARED_DATA.stderr}")


Preparing training data...
Analyzing...

- Your file contains 3 prompt-completion pairs. In general, we recommend having at least a few hundred examples. We've found that performance tends to linearly increase for every doubling of the number of examples
- All prompts end with suffix `ation>`
- All prompts start with prefix `<prompt for `
Failed to analyze training data.
Error: 

ERROR in common_suffix validator: All completions are identical: `<ideal generated bullet>`
Ensure completions are different, otherwise the model will just repeat `<ideal generated bullet>`

Aborting...


## Step 3: Provide Open AI API Key

The OpenAI API key will be taken from your environment.

In [20]:
import os

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
MASKED_OPENAI_API_KEY = "*" * len(OPENAI_API_KEY) if OPENAI_API_KEY else "Error: No key found!"

print("API_KEY:", MASKED_OPENAI_API_KEY)

API_KEY: ***********


## Step 4: Start the Fine-Tuning Job

Once the request is made to the API, the fine-tuning may take a while to complete. The following will check for completion periodically.

In [23]:
import openai

print("==> Starting fine-tuning job...")
if not BASE_MODEL and PREPARED_DATA.returncode == 0:
    exit
print("==> Fine-tuning job started.")

response = openai.FineTune.create(
  model=BASE_MODEL,
  training_data=JSONL_FILE
)

print("==> Fine-tuning in-progress...")
if response.status_code == 200:
  print(f"Fine-tuned Model ID: {response.json()['id']}")
else:
  print(f"Error Occurred: {response.json()['error']}")
  exit

print("==> Fine-tuning completed!")

==> Starting fine-tuning job...
==> Fine-tuning job started.


AttributeError: module 'openai' has no attribute 'FineTuning'

## Step 5: Use The Fine-Tuned Model

Fine-tuned model usage: 

```bash
openai api completions.create -m <FINE_TUNED_MODEL> -p <YOUR PROMPT>
```
